# Tempus Bioinformatics Challenge

This jupyter notebook is demonstrating the use of the accompanying scripts written to parse the .vcf file format and interact with the ExAC API. 

In [1]:
#---- supporting .py files ----
import parseVCF
import ExAC

#---- External Libraries ----
import pandas as pd
import numpy as np

## Parsing VCF

In [2]:
vcf_df = parseVCF.parseVCFtoDataFrame('Challenge_data.vcf')
parseVCF.prioritizeLabel(vcf_df)
parseVCF.calculateVarFrac(vcf_df)

__We now have a full annotated dataframe, here are the desired annotations. __

  - 'CHROM' - Chromosome
  - 'POS' - Position in Chromosome
  - 'DP' - Depth of reads at locus
  - 'REF' - Reference bases
  - 'RO' - Observations of reference bases
  - 'VAR' - Selected Variant based on prioritization of (1) 'complex', (2) 'ins', (3) 'del', (4) 'mnp', (5) 'snp'
  - 'VAR_TYPE'- Annotated variant type based off above priority
  - 'VAR_COUNT' - Count of times selected variant was observed
  - 'VAR_FRAC' - Fraction of total reads that the variant was observed
  
  

In [3]:
display_columns = ['CHROM', 'POS', 'DP', 'REF', 'RO', 'VAR', 'VAR_TYPE', 'VAR_COUNT', 'VAR_FRAC']
vcf_df[display_columns].head(15)

,CHROM,POS,DP,REF,RO,VAR,VAR_TYPE,VAR_COUNT,VAR_FRAC
0,1,931393,4124,G,4029,T,snp,95,0.023036
1,1,935222,1134,C,480,A,snp,652,0.574956
2,1,1277533,786,T,0,C,snp,786,1.000000
3,1,1284490,228,G,0,A,snp,228,1.000000
4,1,1571850,4055,G,3961,A,snp,94,0.023181
5,1,1572579,3456,A,3430,G,snp,26,0.007523
6,1,1575616,1118,T,0,C,snp,1118,1.000000
7,1,1575784,3476,C,3206,T,snp,266,0.076525
8,1,1577180,90,C,40,T,snp,50,0.555556
9,1,1635004,4076,T,0,C,snp,4076,1.000000


In [4]:
print('Dataframe has a size of {0} and the keys: [{1}]'.format(vcf_df.shape, ','.join(list(vcf_df))))

Dataframe has a size of (6977, 12) and the keys: [CHROM,POS,REF,ALT,TYPE,DP,RO,AO,VAR_TYPE,VAR,VAR_COUNT,VAR_FRAC]


## Using ExAC database API

In [5]:
ExACjson = ExAC.callBulkMethod(vcf_df)
ExAC.getFreq(vcf_df, ExACjson)

In [6]:
display_columns = ['CHROM', 'POS', 'DP', 'REF', 'VAR', 'VAR_TYPE', 'FREQ_ExAC']
vcf_df[display_columns].head(15)

,CHROM,POS,DP,REF,VAR,VAR_TYPE,FREQ_ExAC
0,1,931393,4124,G,T,snp,0.233772
1,1,935222,1134,C,A,snp,.
2,1,1277533,786,T,C,snp,.
3,1,1284490,228,G,A,snp,0.36218
4,1,1571850,4055,G,A,snp,0.638381
5,1,1572579,3456,A,G,snp,.
6,1,1575616,1118,T,C,snp,0.507712
7,1,1575784,3476,C,T,snp,.
8,1,1577180,90,C,T,snp,.
9,1,1635004,4076,T,C,snp,.


### Extending the VCF annotations using ExAC API

Part of the coding challenge was to annotate each type of variation for each variant. Part of this specifically asked for variations like "Substitution", "Insertion", "Silent", "Intergenic", etc. 

The problem with using solely the VCF file to annotate with this level of specificity, it would require knowing information like the reading frame, intergenic, which would be found in the surrounding bases and not just the variant. One way that I started trying to circumvent that problem was to look at the associated alleles in ExAC database and the "major consequence" of each variant. This code starts that process, but unfortunately not all of the variants are going to be in the database, and there's not going to be a one-to-one association of VCF annotation to ExAC consequence. 

In [7]:
ExAC.getConsequence(vcf_df, ExACjson)
display_columns = ['CHROM', 'POS', 'DP', 'REF', 'VAR', 'VAR_TYPE', 'TYPE_vep']

vcf_df[display_columns].head(15)

,CHROM,POS,DP,REF,VAR,VAR_TYPE,TYPE_vep
0,1,931393,4124,G,T,snp,missense_variant
1,1,935222,1134,C,A,snp,.
2,1,1277533,786,T,C,snp,.
3,1,1284490,228,G,A,snp,synonymous_variant
4,1,1571850,4055,G,A,snp,synonymous_variant
5,1,1572579,3456,A,G,snp,.
6,1,1575616,1118,T,C,snp,synonymous_variant
7,1,1575784,3476,C,T,snp,.
8,1,1577180,90,C,T,snp,.
9,1,1635004,4076,T,C,snp,.


#### What types of annotations are present in the ExAC database (where I used '.' as a place holder for the ones that were not found - per VCF convention)?

In [8]:
print(' | '.join(vcf_df['TYPE_vep'].unique()) )

missense_variant | . | synonymous_variant | 3_prime_UTR_variant | intron_variant | splice_region_variant | 5_prime_UTR_variant | non_coding_transcript_exon_variant | stop_lost | stop_gained | stop_retained_variant | splice_acceptor_variant | splice_donor_variant | initiator_codon_variant


#### Represent these consequences as percentages of the total population of VCF annotations:

In [9]:
consequenceType = vcf_df.groupby(['VAR_TYPE','TYPE_vep'])['CHROM'].count()

Annotations = pd.DataFrame(consequenceType.values, columns = ['Count'], index = consequenceType.index)

In [10]:
Annotations.reset_index(inplace=True)
AnnotationTable = Annotations.pivot(index = 'VAR_TYPE', columns='TYPE_vep', values='Count')
AnnotationTable = AnnotationTable.fillna(0)
AnnotationTable = AnnotationTable.astype(int)
array = AnnotationTable.values
row_sums = array.sum(axis=1)
new_matrix = array / row_sums[:, np.newaxis]
new_columns = columns=AnnotationTable.columns
PercentAnnotation = pd.DataFrame(new_matrix, index=AnnotationTable.index, columns=AnnotationTable.columns)

In [11]:
PercentAnnotation.head()

TYPE_vep,.,3_prime_UTR_variant,5_prime_UTR_variant,initiator_codon_variant,intron_variant,missense_variant,non_coding_transcript_exon_variant,splice_acceptor_variant,splice_donor_variant,splice_region_variant,stop_gained,stop_lost,stop_retained_variant,synonymous_variant
VAR_TYPE,,,,,,,,,,,,,,
complex,0.577586,0.008621,0.000000,0.000000,0.094828,0.129310,0.008621,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.163793
del,0.523481,0.006906,0.015193,0.001381,0.103591,0.120166,0.009669,0.001381,0.000000,0.029006,0.002762,0.002762,0.001381,0.182320
ins,0.520776,0.004155,0.012465,0.000000,0.108033,0.119114,0.011080,0.000000,0.001385,0.018006,0.000000,0.000000,0.000000,0.204986
mnp,0.315789,0.026316,0.026316,0.000000,0.131579,0.236842,0.026316,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.184211
snp,0.515157,0.008183,0.009671,0.000186,0.111028,0.135577,0.013018,0.000000,0.000000,0.031244,0.000930,0.000186,0.000000,0.174819


__As could have been expected, many of these variants in this file were not identified in the database. About 20% of all the variants across all vcf annotations were silent variants. This could be a first step in trying to make more specific annotations to the vcf file.__




### Make a new VCF file with additional annotations after this analysis

In [12]:
import annotateVCF
annotateVCF.annotate('Challenge_data.vcf',vcf_df)